<h1>CITYBIKE KATOWICE 2018

<h3> You will find below the complete step by step data analysis of the data set from bike provider in Katowice in 2018

In [76]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/FUtOLh4CmsXZe/giphy.gif">')

<h2> First things first

<h3>Import the dataset, review it, add new columns so your work would be easier.

In [5]:
#this will be needed
import pandas as pd
import numpy as np
import datetime

In [6]:
#import dataset
wyp2018 = pd.read_csv('./dane/wyp_2018.csv')

In [7]:
#give it a glimpse
wyp2018.head(5)

,bike_num,start_time,end_time,departure,return,duration_sec
0,58388,2018-04-01 00:16:10,2018-04-01 00:20:17,Murapol Mariacka,KTBS – Krasińskiego 14,247
1,58745,2018-04-01 00:10:28,2018-04-01 00:20:28,Murapol Mariacka,KTBS – Krasińskiego 14,600
2,58547,2018-04-01 10:34:09,2018-04-01 10:41:38,KTBS – Krasińskiego 14,Katowice Rynek,449
3,58786,2018-04-01 11:31:14,2018-04-01 11:45:20,Bogucice Szpital,KTBS – Saint Etienne 1,846
4,58884,2018-04-01 11:29:51,2018-04-01 11:45:39,Bogucice Szpital,KTBS – Saint Etienne 1,948


In [8]:
# check its size
wyp2018.shape

(193328, 6)

In [9]:
#and data types
wyp2018.dtypes

bike_num         int64
start_time      object
end_time        object
departure       object
return          object
duration_sec     int64
dtype: object

In [10]:
#create additional columns dividing departure time into smaller chunks
wyp2018['start_day'] = pd.DatetimeIndex(wyp2018['start_time']).day
wyp2018['start_month'] = pd.DatetimeIndex(wyp2018['start_time']).month
wyp2018['start_hour'] = pd.DatetimeIndex(wyp2018['start_time']).hour
wyp2018['start_minute'] = pd.DatetimeIndex(wyp2018['start_time']).minute
wyp2018['month_day'] = pd.DatetimeIndex(wyp2018['start_time']).date
wyp2018['duration_min'] = wyp2018['duration_sec']/60
wyp2018['duration_hour'] = (wyp2018['duration_min']/60).apply(int)

#create additional columns to gather more information. Which day of week was it? monday = 0, sunday = 6
wyp2018['which_day'] = pd.DatetimeIndex(wyp2018['start_time']).dayofweek

# Which day of year was it?
wyp2018['daynumber'] = pd.DatetimeIndex(wyp2018['start_time']).dayofyear

#was it weekend?
wyp2018['is_weekend'] = wyp2018['which_day'].apply(lambda x: 1 if (x == 6 or x == 5 ) else 0 )

#was it a holiday?
free = [ datetime.date(2018,1,1),
        datetime.date(2018,1,6),
        datetime.date(2018,4,1),
        datetime.date(2018,4,2),
        datetime.date(2018,5,1),
        datetime.date(2018,5,3),
        datetime.date(2018,5,20),
        datetime.date(2018,5,31),
        datetime.date(2018,8,15),
        datetime.date(2018,11,1),
        datetime.date(2018,11,11),
        datetime.date(2018,11,12),
        datetime.date(2018,12,25),
        datetime.date(2018,12,26)
        ]

wyp2018['is_free'] = wyp2018['month_day'].apply(lambda x: 1 if (x in free) else 0 )


#was it a shopping sunday?
handl = [datetime.date(2018,3,25),
        datetime.date(2018,4,29),
        datetime.date(2018,5,27),
        datetime.date(2018,6,24),
        datetime.date(2018,7,29),
        datetime.date(2018,8,26),
        datetime.date(2018,9,30),
        datetime.date(2018,10,28),
        datetime.date(2018,11,25),
        datetime.date(2018,12,16),
        datetime.date(2018,12,23)
        ]

def conditions(s):
    if (s['month_day'] in handl):
        return 1
    elif (s['month_day'] not in handl) and (s['which_day'] == 6):
        return 0
    else:
        return 2
    
wyp2018['shop'] = wyp2018.apply(conditions, axis=1)

#create additional column to keep the route information in one place (departure - return)
wyp2018["route"] = wyp2018["departure"] + ' - ' + wyp2018["return"]

<h2>Remove the data that is not necessary.

In [11]:
#What do we know for sure?
print('Bikes were available for rent for {0} days.'.format(wyp2018['month_day'].nunique()))
print('Bikes were available for rent for {0} months.'.format(wyp2018['start_month'].nunique()))

Bikes were available for rent for 260 days.
Bikes were available for rent for 9 months.


In [12]:
#That is all we can be sure of. Before we start to look for number of rents, average rent time etc,
#we need to check the data looking for outliers, odd values, anything that looks suspicious.

In [13]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/NS7gPxeumewkWDOIxi/giphy.gif">')

In [14]:
#Let's start with the time of rentals and see how long are they. Are any of them too short to be true? 
(wyp2018.assign(minutes = lambda _wyp2018: _wyp2018['duration_min'].astype(int))
        .groupby('minutes')
        .count()
        .head(5)
)['bike_num']

minutes
0    11204
1     6488
2     4448
3     5864
4     7756
Name: bike_num, dtype: int64

In [15]:
#Google Maps states you can ride 400m in 1 minute. I think we can safely assume that if a bike was returned to the
#same station it was rented in less than 3 minutes, user has change his mind or the bike was broken. Let's remove
#from the dataset all the rides that lasted lesss than 3 minutes and have the depart = return.
#It will be 20363 records
remove = wyp2018[(wyp2018['duration_min'] < 3) & (wyp2018['departure'] == wyp2018['return'])].index
wyp2018 = wyp2018.drop(index=remove)

In [16]:
#Now let's check all the rides that lasted less than 3 minutes, but depart is different than return. We got 1777 of them
wyp2018[(wyp2018['duration_min'] < 3) &  (wyp2018['departure'] != wyp2018['return'])].head(3)

,bike_num,start_time,end_time,departure,return,duration_sec,start_day,start_month,start_hour,start_minute,month_day,duration_min,duration_hour,which_day,daynumber,is_weekend,is_free,shop,route
33,58517,2018-04-01 18:28:43,2018-04-01 18:31:42,Silesia City Center,Murapol Chorzowska,179,1,4,18,28,2018-04-01,2.983333,0,6,91,1,1,0,Silesia City Center - Murapol Chorzowska
282,58882,2018-04-02 19:09:05,2018-04-02 19:11:40,Politechnika Śląska,KTBS – Krasińskiego 14,155,2,4,19,9,2018-04-02,2.583333,0,0,92,0,1,2,Politechnika Śląska - KTBS – Krasińskiego 14
315,58822,2018-04-02 22:06:10,2018-04-02 22:07:01,Silesia Business Park,Murapol Mariacka,51,2,4,22,6,2018-04-02,0.850000,0,0,92,0,1,2,Silesia Business Park - Murapol Mariacka


In [17]:
#Some of them are reasonable, for example Politechnika Śląska	KTBS – Krasińskiego 14. The distance between those 
#2 stations is 230m. I propose to count distance for every route in the dataset, and than remove all the records
#that are shorter than 3 minutes AND the distance between them is greater that 1200m (1 min = 400m according to Google Maps)

In [18]:
#To calculate distance between the stations we need the geocoordinates,thus we will upload stations dataset
#and merge with wyp2018
stations = pd.read_csv('dane/stacje_2018.csv')
#merge datasets adding geo coordinates to departure station
df_temp = pd.merge(wyp2018, stations, left_on='departure', right_on='name')
#merge datasets adding geo coordinates to return station
wyp2018 = pd.merge(df_temp, stations, left_on='return', right_on='name')
#remove unnecessary columns, rename ambigous
wyp2018 = wyp2018.drop(columns = ['name_x', 'name_y', 'capacity_x', 'capacity_y'])
wyp2018 = wyp2018.rename(columns={'id_x': 'dep_id', 'lat_x': 'dep_lat', 'lon_x' : 'dep_lon', 'id_y' : 'ret_id',
                                  'lat_y' : 'ret_lat', 'lon_y' : 'ret_lon'})
#sort df by 'start_time' column
wyp2018 = wyp2018.sort_values(by=['start_time'])

In [19]:
#for each route, calculate the distance. We only can calculate the distance of a straight line between the departure and
#return points.
from geopy.distance import geodesic
wyp2018['distance'] = (wyp2018[['dep_lat','dep_lon','ret_lat','ret_lon']]
        .apply(lambda x: geodesic( [ x[0],x[1] ], [ x[2],x[3] ] ).km, axis=1))

In [20]:
#find all the records that are shorter than 3 minutes AND the distance between them is greater that 1200m
#there are 186 of them - let's remove them
remove = wyp2018[(wyp2018['duration_min'] < 3) & (wyp2018['distance'] > 1.2)].index
wyp2018 = wyp2018.drop(index=remove)

In [21]:
#we have removed 20549 records, what makes 10,6% of the whole dataset

In [22]:
#Let's then check oddly long rentals by grouping rentals by hours
(wyp2018
         .groupby('duration_hour')['bike_num']
         .count()
)

duration_hour
0     145343
1      17859
2       6002
3       2196
4        741
5        254
6        117
7         59
8         32
9         29
10        22
11        22
12        25
13         9
14        13
15         4
16         5
17         2
18         1
19         4
20         4
21         2
22         4
23         4
24         2
25         2
26         5
28         1
31         3
35         1
37         1
40         1
41         2
45         1
46         2
47         1
48         1
49         1
55         1
69         1
Name: bike_num, dtype: int64

In [23]:
#the MAJORITY of rentals last less than 1 hour. But there are some REALLY STRANGE ones lasting even more than 24 hours
#How many rentals are more than 3 hours?
wyp2018[wyp2018['duration_hour'] > 3].count().head(1)

bike_num    1379
dtype: int64

In [24]:
#That is merely a 0.1% of whole dataset
(wyp2018[wyp2018['duration_hour'] > 3].count()/wyp2018['duration_hour'].count()).head(1)

bike_num    0.007981
dtype: float64

In [25]:
#I assumed we can remove all the rentals that lasted more than 3 hours without causing a damage to the results
remove = wyp2018[wyp2018['duration_hour'] > 3].index
wyp2018 = wyp2018.drop(index=remove)

In [26]:
#The dataframe is now of shape:
wyp2018.shape

(171400, 26)

<h2>Once you have nearly all the data, let's start with basic analysis.

In [27]:
#Now that the outliers are gone we can start analyzing data, starting with the number of bikes and rentals in total
print('There were {0} bikes around the town.'.format(wyp2018['bike_num'].nunique()))
print('They were rented {0} times in total.'.format(wyp2018['bike_num'].count()))

There were 420 bikes around the town.
They were rented 171400 times in total.


In [28]:
#Also, the average time of rentals:
wyp2018['duration_min'].mean()

30.96407477635108

In [29]:
#the median of rentals' time
wyp2018['duration_min'].median()

13.283333333333333

In [30]:
#If the rental was less than 15 minutes, it was free of charge
print('Number of rentals that were free (less than 15 minutes): ')
wyp2018[wyp2018['duration_min'] < 15]['bike_num'].count()

Number of rentals that were free (less than 15 minutes): 


93823

In [31]:
print('It was {0:.2f} of total'.format(wyp2018[wyp2018['duration_min'] < 15]['bike_num'].count()/(wyp2018['bike_num'].count())))

It was 0.55 of total


<h3>Now it is time for more specific information - number of rentals per month/day/etc

In [32]:
#Like the number of rental in each month:
print('Number of rentals per month:')
print(wyp2018.groupby('start_month')['bike_num'].count())
print('Most renatls in: ', (wyp2018
                            .groupby('start_month')['bike_num']
                            .count()
                            .nlargest(1)))

Number of rentals per month:
start_month
4     24479
5     26022
6     28337
7     26644
8     29213
9     17235
10    12098
11     6215
12     1157
Name: bike_num, dtype: int64
Most renatls in:  start_month
8    29213
Name: bike_num, dtype: int64


In [33]:
#Like the number of rentals per day of a week:
print('rental number per day of a week: ')
print(wyp2018.groupby('which_day')['bike_num'].count())
print('Most renatls in: ', (wyp2018
                            .groupby('which_day')['bike_num']
                            .count()
                            .nlargest(1)))

rental number per day of a week: 
which_day
0    24716
1    24388
2    25565
3    24403
4    23941
5    23464
6    24923
Name: bike_num, dtype: int64
Most renatls in:  which_day
2    25565
Name: bike_num, dtype: int64


In [34]:
#the average rentals' time per week day
print('average rentals time per week day:')
print((wyp2018
       .groupby('which_day')['duration_min']
       .mean()))

average rentals time per week day:
which_day
0    26.052410
1    25.479521
2    26.388095
3    27.696184
4    25.510821
5    38.703966
6    47.046923
Name: duration_min, dtype: float64


In [35]:
#the median of rentals time per week day
print('median of rentals time per week day:')
print((wyp2018
       .groupby('which_day')['duration_min']
       .median()))

median of rentals time per week day:
which_day
0    11.966667
1    11.766667
2    11.866667
3    12.250000
4    11.933333
5    18.050000
6    25.950000
Name: duration_min, dtype: float64


In [36]:
#on what day was there the most rentals?
print('Days with most rentals:  ')
(wyp2018
     .groupby('month_day')['bike_num']
     .count()
     .nlargest(5))

Days with most rentals:  


month_day
2018-06-20    1317
2018-08-15    1308
2018-06-06    1304
2018-06-07    1253
2018-05-31    1246
Name: bike_num, dtype: int64

In [37]:
print('Most popular routes on 15 August: ')
(wyp2018[wyp2018['month_day'] == datetime.date(2018, 8, 15)]
        .groupby('route')['bike_num']
        .count()
        .nlargest(5))


Most popular routes on 15 August: 


route
Dolina 3-ch Stawów - Dolina 3-ch Stawów                          56
Katowice Rynek - Katowice Rynek                                  51
Al. Bolesława Krzywoustego - Al. Bolesława Krzywoustego          27
Al. Księżnej Jadwigi Śląskiej - Al. Księżnej Jadwigi Śląskiej    21
KTBS – Saint Etienne 1 - KTBS – Saint Etienne 1                  18
Name: bike_num, dtype: int64

In [38]:
#On which days during popular months were there the least rentals?
(wyp2018[wyp2018['start_month']
         .isin([5,6,7,8])]
         .groupby('month_day')['bike_num']
         .count().nsmallest(5))

month_day
2018-05-17    105
2018-07-18    245
2018-07-17    300
2018-06-28    380
2018-05-18    387
Name: bike_num, dtype: int64

In [39]:
#how many holidays were there?
(wyp2018[wyp2018['is_free'] == 1]
     .groupby('month_day')
     .count()
     .index
     .nunique())

10

In [40]:
#how many rentals were there?
wyp2018[wyp2018['is_free'] == 1]['bike_num'].count()

6596

In [41]:
#what percent of total is this?
print('It is {0:.2f} of total'.format(wyp2018[wyp2018['is_free'] == 1]['bike_num'].count()/(wyp2018['bike_num'].count())))

It is 0.04 of total


In [42]:
#how many rentals were there?
print('Rentals on holidays: ')
(wyp2018[wyp2018['is_free'] == 1]
     .groupby('month_day')['bike_num']
     .count()
     .nlargest(5))

Rentals on holidays: 


month_day
2018-08-15    1308
2018-05-31    1246
2018-05-01    1172
2018-05-20     927
2018-05-03     840
Name: bike_num, dtype: int64

In [43]:
#Let's now compare the averages together
#comparing the number of rentals during working days, weekends and holidays
print('average number of rentals during working days: ',
      wyp2018[wyp2018['is_weekend'] == 0]['bike_num'].count() / wyp2018[wyp2018['which_day'].isin(range(0,5))]['month_day'].nunique())
print('average number of rentals during weekendss: ',
      wyp2018[wyp2018['is_weekend'] == 1]['bike_num'].count() / wyp2018[wyp2018['which_day'].isin([5,6])]['month_day'].nunique())
print('average number of rentals during holidays: ',
      wyp2018[wyp2018['is_free'] == 1]['bike_num'].count() / wyp2018[wyp2018['is_free'] == 1].groupby('month_day').count().index.nunique())

average number of rentals during working days:  664.9351351351352
average number of rentals during weekendss:  645.16
average number of rentals during holidays:  659.6


In [44]:
#Let's now compare the average time of rentals together
#comparing the average time of rentals during working days, weekends and holidays
print('average time of rentals during working days: ',
      wyp2018[wyp2018['is_weekend'] == 0]['duration_min'].mean())
print('average time of rentals during weekends: ',
      wyp2018[wyp2018['is_weekend'] == 1]['duration_min'].mean())
print('average time of rentals during holidays: ',
      wyp2018[wyp2018['is_free'] == 1]['duration_min'].mean())

average time of rentals during working days:  26.229277528933192
average time of rentals during weekends:  43.00122553578421
average time of rentals during holidays:  50.65119264200518


In [45]:
#We can also check what is happening durind the Sundays with shops closed vs Sundays when shopping is possible

#number of sundays you can shop
lh = (wyp2018[wyp2018['shop'] == 1]
      .groupby('month_day')['bike_num']
      .nunique()
      .count())

#number of sundays you canot shop
lnh = (wyp2018[wyp2018['shop'] == 0]
       .groupby('month_day')['bike_num']
       .nunique()
       .count())

print('number of sundays you can shop: ', lh)
print('number of sundays you cannot shop: ',  lnh)

print('number of rentals from sundays you can shop: ', wyp2018[wyp2018['shop'] == 1]['bike_num'].count())
print('number of rentals from sundays you cannot shop: ', wyp2018[wyp2018['shop'] == 0]['bike_num'].count())

print('average number of rentals from sundays you can shop: ', wyp2018[wyp2018['shop'] == 1]['bike_num'].count() / lh)
print('average number of rentals from sundays you cannot shop: ', wyp2018[wyp2018['shop'] == 0]['bike_num'].count() / lnh)

print('average time of rentals from sundays you can shop: ', wyp2018[wyp2018['shop'] == 1]['duration_min'].mean())
print('average time of rentals from sundays you cannot shop: ', wyp2018[wyp2018['shop'] == 0]['duration_min'].mean())

number of sundays you can shop:  9
number of sundays you cannot shop:  29
number of rentals from sundays you can shop:  3811
number of rentals from sundays you cannot shop:  21112
average number of rentals from sundays you can shop:  423.44444444444446
average number of rentals from sundays you cannot shop:  728.0
average time of rentals from sundays you can shop:  38.641800052479574
average time of rentals from sundays you cannot shop:  48.564160193255255


<h3>ALso, let's see how bikes are rented during the day.

In [46]:
#number of working days the bikes were rented
lr = (wyp2018[wyp2018['which_day']
              .isin(range(0,5))]['month_day']
              .nunique())
#number of weekend days the bikes were rented
lw = (wyp2018[wyp2018['which_day']
              .isin([5,6])]['month_day']
              .nunique())
#number of holidays the bikes were rented
sw = (wyp2018[wyp2018['is_free'] == 1]
              .groupby('month_day')
              .count()
              .index
              .nunique())

#sum of rentals by hour for working days
a = (wyp2018[wyp2018['is_weekend'] == False]
     .groupby('start_hour')['bike_num']
     .count())
#sum of rentals by hour for weekend days
b = (wyp2018[wyp2018['is_weekend'] == True]
     .groupby('start_hour')['bike_num']
     .count())
#sum of rentals by hour for holidays
s = (wyp2018[wyp2018['is_free'] == True]
     .groupby('start_hour')['bike_num']
     .count())

#average number of rentals by hour for working days
c = a / lr
#average number of rentals by hour for weekend days
d = b / lw
#srednia liczba wypozyczen dla przedzialow godzinowych w  dni świąteczne wolne od pracy
t = s / sw

#average time of rentals by hour for working days
e = wyp2018[wyp2018['is_weekend'] == False].groupby('start_hour')['duration_min'].sum() / a
#average time of rentals by hour for weekend days
f = wyp2018[wyp2018['is_weekend'] == True].groupby('start_hour')['duration_min'].sum() / b
#average time of rentals by hour for holidays
u = wyp2018[wyp2018['is_free'] == True].groupby('start_hour')['duration_min'].sum() / s

hour_data = {'sum working days' : a.values, 'sum weekend' : b.values, 'sum holidays' : s.values,
             'mean working' : c.values, 'mean weekend' : d.values, 'mean holidays' : t.values,
             'mean time working' : e.values, 'mean time weekend' : f.values, 'mean time holidays' : u.values}

hourdf = pd.DataFrame(data=hour_data)
hourdf.head(5)

,sum working days,sum weekend,sum holidays,mean working,mean weekend,mean holidays,mean time working,mean time weekend,mean time holidays
0,1037,972,112,5.605405,12.96,11.2,19.402636,22.766032,17.857143
1,620,711,72,3.351351,9.48,7.2,24.354516,18.026512,19.511806
2,355,498,36,1.918919,6.64,3.6,18.623333,19.315529,18.177778
3,314,354,34,1.697297,4.72,3.4,15.067091,18.786158,15.368137
4,237,183,10,1.281081,2.44,1.0,16.058087,16.296266,11.611667


In [47]:
#which station are popular during which hours?

#popular routes during working days
t = wyp2018[wyp2018['is_weekend'] == False].groupby(['start_hour','route'])['bike_num'].count()
trasy = []
for i in range(0,24):
    trasy.append(t.loc[i].sort_values(ascending=False).index[0])

#popular routes during weekend days
t = wyp2018[wyp2018['is_weekend'] == True].groupby(['start_hour','route'])['bike_num'].count()
trasyw = []
for i in range(0,24):
    trasyw.append(t.loc[i].sort_values(ascending=False).index[0])
    
#popular routes during holidays
t = wyp2018[wyp2018['is_free'] == True].groupby(['start_hour','route'])['bike_num'].count()
trasyh = []
for i in range(0,24):
    trasyh.append(t.loc[i].sort_values(ascending=False).index[0])

trasy_data = {'working_days' : trasy, 'weekends' : trasyw, 'holidays': trasyh}

trasy_data = pd.DataFrame(data=trasy_data)

trasy_data

,working_days,weekends,holidays
0,Murapol Mariacka - KTBS – Krasińskiego 14,Katowice Rynek - Katowice Rynek,Katowice Rynek - Murapol Mariacka
1,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Saint Etienne 1
2,KTBS – Krasińskiego 14 - Katowice Rynek,Katowice Rynek - Katowice Rynek,Katowice Rynek - Katowice Rynek
3,Kostuchna – Szarych Szeregów - Kostuchna – Ron...,Katowice Rynek - Katowice Rynek,Katowice Rynek - Katowice Rynek
4,KTBS – Krasińskiego 14 - Katowice Rynek,Katowice Rynek - Murapol Chorzowska,Łętowskiego S.P. nr 27 - Piotrowice V L.O.
5,Łętowskiego S.P. nr 27 - Zadole Kościół,Łętowskiego S.P. nr 27 - Zadole Kościół,Murapol Mariacka - Bogucice Szpital
6,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek,Dolina 3-ch Stawów - Dolina 3-ch Stawów
7,Al. Bolesława Krzywoustego - Silesia Business ...,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek
8,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek
9,Katowice Rynek - Politechnika Śląska,Katowice Rynek - Silesia City Center,Al. Księcia Henryka Pobożnego - Al. Księcia He...


In [48]:
#we can check the same thing, but exclude the routes where departure = return

#popular routes during working days
t = wyp2018[(wyp2018['is_weekend'] == False) & (wyp2018['departure'] != wyp2018['return'])].groupby(['start_hour','route'])['bike_num'].count()
trasy = []
for i in range(0,24):
    trasy.append(t.loc[i].sort_values(ascending=False).index[0])

#popular routes during weekend days
t = wyp2018[(wyp2018['is_weekend'] == True) & (wyp2018['departure'] != wyp2018['return'])].groupby(['start_hour','route'])['bike_num'].count()
trasyw = []
for i in range(0,24):
    trasyw.append(t.loc[i].sort_values(ascending=False).index[0])
    
#popular routes during holidays
t = wyp2018[(wyp2018['is_free'] == True) & (wyp2018['departure'] != wyp2018['return'])].groupby(['start_hour','route'])['bike_num'].count()
trasyh = []
for i in range(0,24):
    trasyh.append(t.loc[i].sort_values(ascending=False).index[0])

trasy_data = {'working_days' : trasy, 'weekends' : trasyw, 'holidays': trasyh}

trasy_data = pd.DataFrame(data=trasy_data)

trasy_data

,working_days,weekends,holidays
0,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Krasińskiego 14,Katowice Rynek - Murapol Mariacka
1,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Saint Etienne 1
2,KTBS – Krasińskiego 14 - Katowice Rynek,Murapol Mariacka - KTBS – Krasińskiego 14,COP24 - Murapol Mariacka
3,Kostuchna – Szarych Szeregów - Kostuchna – Ron...,Katowice Rynek - Plac Wolności,KTBS – Krasińskiego 14 - Murapol Mariacka
4,KTBS – Krasińskiego 14 - Katowice Rynek,Katowice Rynek - Murapol Chorzowska,Łętowskiego S.P. nr 27 - Piotrowice V L.O.
5,Łętowskiego S.P. nr 27 - Zadole Kościół,Łętowskiego S.P. nr 27 - Zadole Kościół,Murapol Mariacka - Bogucice Szpital
6,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek,Zadole Kościół - Ligota Dworzec PKP
7,Al. Bolesława Krzywoustego - Silesia Business ...,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek
8,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek
9,Katowice Rynek - Politechnika Śląska,Katowice Rynek - Silesia City Center,Załęże Skwer S. Barei - Plac Wolności


In [49]:
#it seemed strange for me that there are not so many rides through parks during the weekends, so let's
#compare saturdays, shopping sundays and not-shopping sundays


#popular routes during saturdays
t = wyp2018[(wyp2018['which_day'] == 5)].groupby(['start_hour','route'])['bike_num'].count()
trasy = []
for i in range(0,24):
    trasy.append(t.loc[i].sort_values(ascending=False).index[0])

#popular routes during sundays shopping
t = wyp2018[(wyp2018['which_day'] == 6) & wyp2018['shop'] == 1].groupby(['start_hour','route'])['bike_num'].count()
trasyw = []
for i in range(0,24):
    trasyw.append(t.loc[i].sort_values(ascending=False).index[0])
    
#popular routes during sundays not shopping
t = wyp2018[(wyp2018['which_day'] == 6) & wyp2018['shop'] == 0].groupby(['start_hour','route'])['bike_num'].count()
trasyn = []
for i in range(0,24):
    trasyn.append(t.loc[i].sort_values(ascending=False).index[0])
    
#popular routes during holidays
t = wyp2018[(wyp2018['is_free'] == True)].groupby(['start_hour','route'])['bike_num'].count()
trasyh = []
for i in range(0,24):
    trasyh.append(t.loc[i].sort_values(ascending=False).index[0])

trasy_data = {'saturdays' : trasy, 'sundays' : trasyw, 'sundays_no_shop' : trasyn, 'holidays': trasyh}

trasy_data = pd.DataFrame(data=trasy_data)

trasy_data


,saturdays,sundays,sundays_no_shop,holidays
0,Murapol Mariacka - KTBS – Krasińskiego 14,Katowice Rynek - Katowice Rynek,Murapol Mariacka - KTBS – Krasińskiego 14,Katowice Rynek - Murapol Mariacka
1,Murapol Mariacka - KTBS – Krasińskiego 14,Al. Księcia Henryka Pobożnego - Katowice Rynek,Murapol Mariacka - KTBS – Krasińskiego 14,Murapol Mariacka - KTBS – Saint Etienne 1
2,Katowice Rynek - Katowice Rynek,Katowice Rynek - Park Kościuszki,Katowice Rynek - Katowice Rynek,Katowice Rynek - Katowice Rynek
3,Katowice Rynek - Katowice Rynek,Al. Księcia Henryka Pobożnego - Al. Księcia He...,Kostuchna – Szarych Szeregów - Kostuchna – Ron...,Katowice Rynek - Katowice Rynek
4,Kostuchna – Szarych Szeregów - Kostuchna – Ron...,Murapol Mariacka - Murapol Mariacka,KTBS – Krasińskiego 14 - Katowice Rynek,Łętowskiego S.P. nr 27 - Piotrowice V L.O.
5,Łętowskiego S.P. nr 27 - Zadole Kościół,Murapol Mariacka - Ul. Powstańców - Biblioteka...,Łętowskiego S.P. nr 27 - Zadole Kościół,Murapol Mariacka - Bogucice Szpital
6,KTBS – Krasińskiego 14 - Katowice Rynek,Podlesie – Stary Most - Podlesie – Stary Most,KTBS – Krasińskiego 14 - Katowice Rynek,Dolina 3-ch Stawów - Dolina 3-ch Stawów
7,KTBS – Krasińskiego 14 - Katowice Rynek,Plac Wolności - Załęże Skwer S. Barei,Al. Bolesława Krzywoustego - Silesia Business ...,KTBS – Krasińskiego 14 - Katowice Rynek
8,KTBS – Krasińskiego 14 - Katowice Rynek,Park Kościuszki - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek,KTBS – Krasińskiego 14 - Katowice Rynek
9,Katowice Rynek - Silesia City Center,Katowice Rynek - Silesia City Center,Katowice Rynek - Silesia City Center,Al. Księcia Henryka Pobożnego - Al. Księcia He...


<h3>How about summer holidays? Does students use bikes a lot?

In [50]:
# #Let's see if popular stations varies during summer holidays
# trasy_miesiace = pd.DataFrame()
# for i in range(4,13):
#     t = wyp2018[(wyp2018['start_month'] == i) & (wyp2018['departure'] != wyp2018['return'])].groupby(['start_hour','route'])['bike_num'].count()
#     temp = []
#     for j in range(0,24):
#         temp.append(t.loc[j].sort_values(ascending=False).index[0])
#     trasy_miesiace[i] = temp
#trasy_miesiace

#The above code will show popular stations per hour per month - but I didn't find anything interesting

<h3>Now let's check the stations.

In [51]:
#find the most popular departure stations
print('Most bikes are rented from: ')
wyp2018.groupby('departure')['bike_num'].count().nlargest(5)

Most bikes are rented from: 


departure
Katowice Rynek                27459
Silesia City Center            8730
KTBS – Krasińskiego 14         8378
Murapol Mariacka               7773
Al. Bolesława Krzywoustego     6518
Name: bike_num, dtype: int64

In [52]:
#Does it change througout the year?
stacje_mies_dep = pd.DataFrame()
for i in range(4,13):
    t = wyp2018[wyp2018['start_month'] == i].groupby(['departure'])['bike_num'].count().nlargest(5)
    stacje_mies_dep[i] = t.index
stacje_mies_dep

,4,5,6,7,8,9,10,11,12
0,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek
1,Murapol Mariacka,Murapol Mariacka,Silesia City Center,Silesia City Center,Silesia City Center,Silesia City Center,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14
2,Silesia City Center,Dolina 3-ch Stawów,Murapol Mariacka,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,Silesia City Center,Silesia City Center,Plac Wolności
3,KTBS – Krasińskiego 14,Silesia City Center,KTBS – Krasińskiego 14,Murapol Mariacka,Murapol Mariacka,Plac Wolności,Plac Wolności,Koszutka – Plac Gwarków,Plac Sejmu Śląskiego
4,Al. Księcia Henryka Pobożnego,KTBS – Krasińskiego 14,Plac Wolności,Al. Bolesława Krzywoustego,Al. Bolesława Krzywoustego,Murapol Mariacka,Murapol Mariacka,Plac Sejmu Śląskiego,Murapol Mariacka


In [53]:
#find the most popular return stations
print('Bikes are most returned at: ')
wyp2018.groupby('return')['bike_num'].count().nlargest(5)

Bikes are most returned at: 


return
Katowice Rynek                29073
KTBS – Krasińskiego 14         8230
Murapol Mariacka               8076
Silesia City Center            8075
Al. Bolesława Krzywoustego     7217
Name: bike_num, dtype: int64

In [54]:
#Does it change througout the year?
stacje_mies_ret = pd.DataFrame()
for i in range(4,13):
    t = wyp2018[wyp2018['start_month'] == i].groupby(['return'])['bike_num'].count().nlargest(5)
    stacje_mies_ret[i] = t.index
stacje_mies_ret

,4,5,6,7,8,9,10,11,12
0,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek,Katowice Rynek
1,Murapol Mariacka,Murapol Mariacka,Silesia City Center,Silesia City Center,Silesia City Center,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14
2,Al. Bolesława Krzywoustego,Dolina 3-ch Stawów,Murapol Mariacka,Al. Bolesława Krzywoustego,KTBS – Krasińskiego 14,Silesia City Center,Murapol Mariacka,Plac Wolności,Plac Wolności
3,Al. Księcia Henryka Pobożnego,Al. Bolesława Krzywoustego,KTBS – Krasińskiego 14,KTBS – Krasińskiego 14,Al. Bolesława Krzywoustego,Murapol Mariacka,Plac Wolności,Murapol Mariacka,Murapol Mariacka
4,KTBS – Krasińskiego 14,Al. Księcia Henryka Pobożnego,Al. Bolesława Krzywoustego,Murapol Mariacka,Murapol Mariacka,Plac Wolności,Silesia City Center,Silesia City Center,Ligota Medyków


In [55]:
#Find least popular departure stations:
print('Bikes are least rented at: ')
wyp2018.groupby('departure')['bike_num'].count().nsmallest(5)

Bikes are least rented at: 


departure
PKN Orlen - Al. Roździeńskiego     31
ING Roździeńska                    51
PKN Orlen - Bocheńskiego           83
PKN Orlen - Piotrowicka           201
PKN Orlen - Murckowska            232
Name: bike_num, dtype: int64

In [56]:
#Find the least popular return stations:
print('Bikes are least returned at: ')
wyp2018.groupby('return')['bike_num'].count().nsmallest(5)

Bikes are least returned at: 


return
ING Roździeńska                    27
PKN Orlen - Al. Roździeńskiego     34
PKN Orlen - Bocheńskiego           81
PKN Orlen - Murckowska            143
PKN Orlen - Piotrowicka           185
Name: bike_num, dtype: int64

In [57]:
#Find most popular routes
print('Most popular routes are: ')
wyp2018.groupby('route')['bike_num'].count().nlargest(5)

Most popular routes are: 


route
Katowice Rynek - Katowice Rynek                                  3807
KTBS – Krasińskiego 14 - Katowice Rynek                          3418
Dolina 3-ch Stawów - Dolina 3-ch Stawów                          3253
Katowice Rynek - KTBS – Krasińskiego 14                          3207
Al. Księcia Henryka Pobożnego - Al. Księcia Henryka Pobożnego    2806
Name: bike_num, dtype: int64

In [58]:
#Find most popular routes where departure != return
wyp2018[wyp2018['departure'] != wyp2018['return']].groupby('route')['bike_num'].count().nlargest(5)

route
KTBS – Krasińskiego 14 - Katowice Rynek    3418
Katowice Rynek - KTBS – Krasińskiego 14    3207
Silesia City Center - Katowice Rynek       2288
Katowice Rynek - Murapol Mariacka          1995
Katowice Rynek - Silesia City Center       1797
Name: bike_num, dtype: int64

In [59]:
#Find least popular routes
print('least popular routes are: ')
wyp2018.groupby('route')['bike_num'].count().nsmallest()

least popular routes are: 


route
Al. Bolesława Krzywoustego - Os. Ptasie – ul. Drozdów                1
Al. Bolesława Krzywoustego - Łętowskiego S.P. nr 27                  1
Al. Księcia Henryka Pobożnego - Kostuchna – Rondo Rostworowskiego    1
Al. Księcia Henryka Pobożnego - Ligota Wczasowa                      1
Al. Księcia Henryka Pobożnego - Murcki - Rynek Murckowski            1
Name: bike_num, dtype: int64

<h3>And the most rented bike

In [60]:
#find most rented bike
print('ID of most rented bike and number of its rides in a season: ')
wyp2018.groupby('bike_num')['duration_sec'].count().nlargest(1)

ID of most rented bike and number of its rides in a season: 


bike_num
58463    772
Name: duration_sec, dtype: int64

In [61]:
#find most rented id
mr_bike_id = wyp2018.groupby('bike_num')['start_time'].count().sort_values(ascending=False).head(1).index[0]
#find all its rides
mr_bike = wyp2018[wyp2018['bike_num'] == mr_bike_id]
#find most busy day
date = mr_bike.groupby('month_day')['bike_num'].count().sort_values(ascending=False).index[0]
#create df for this day
mr_bike_day = mr_bike[mr_bike['month_day'] == date]
#sort by start_time
mr_bike_day = mr_bike_day.sort_values(by=['start_time'])
mr_bike_day.head(2)

,bike_num,start_time,end_time,departure,return,duration_sec,start_day,start_month,start_hour,start_minute,...,is_free,shop,route,dep_id,dep_lat,dep_lon,ret_id,ret_lat,ret_lon,distance
35688,58463,2018-04-30 00:19:28,2018-04-30 00:25:28,COP24,Katowice Rynek,360,30,4,0,19,...,0,2,COP24 - Katowice Rynek,5868,50.265282,19.028635,5867,50.258697,19.021618,0.887056
52591,58463,2018-04-30 00:52:42,2018-04-30 00:59:45,Katowice Rynek,COP24,423,30,4,0,52,...,0,2,Katowice Rynek - COP24,5867,50.258697,19.021618,5868,50.265282,19.028635,0.887056


In [72]:
#show most rented bike's busiest day on map

fig = plt.figure()

#starting point
plt.scatter(mr_bike_day.iloc[0]['dep_lon'], mr_bike_day.iloc[0]['dep_lat'], color='purple', s=200)
plt.text(mr_bike_day.iloc[0]['dep_lon'], mr_bike_day.iloc[0]['dep_lat'], 'start', fontsize=12)

#finishing point
plt.scatter(mr_bike_day.iloc[-1]['ret_lon'], mr_bike_day.iloc[-1]['ret_lat'], color='purple', s=200)
plt.text(mr_bike_day.iloc[-1]['ret_lon'], mr_bike_day.iloc[-1]['ret_lat'], 'koniec', fontsize=12)

#rides
plt.plot(mr_bike_day['dep_lon'].values, mr_bike_day['dep_lat'].values)
plt.plot(mr_bike_day['ret_lon'].values, mr_bike_day['ret_lat'].values, marker='o')

# if you are using notebook 
mplleaflet.display(fig=fig)
#mplleaflet.show(fig=fig)

In [62]:
#does the busiest bike visit all the stations during te season?
#find all the stations the bike has visited:
a_stations = mr_bike.groupby('departure').count().index.to_list()
b_stations = mr_bike.groupby('return').count().index.to_list()
stats = list(set(a_stations) | set(b_stations))

#find all the stations available
alls = stations['name'].to_list()

#find the difference
not_visited = (list(set(alls) - set(stats)))
not_visited

['Szopienice - Plac Powstańców Śląskich',
 'Giszowiec - Plac Pod Lipami',
 'PKN Orlen - Piotrowicka',
 'PKN Orlen - Al. Roździeńskiego',
 'Osiedle Franciszkańskie',
 'Os. Ptasie – ul. Drozdów',
 'Kostuchna – Szarych Szeregów',
 'ING Roździeńska',
 'Kostuchna – Boże Dary',
 'PKN Orlen - Bocheńskiego',
 'Kokociniec S.P nr 67',
 'PKN Orlen - Murckowska']

In [63]:
#how many bikes were  moving daily
lr = wyp2018.groupby(['month_day'])['bike_num'].nunique()
lr_data = {'data' : lr.index, 'bikes_moving' : lr.values}
lrdf = pd.DataFrame(data=lr_data)

#vs how many rentals were there
lw = wyp2018.groupby(['month_day'])['bike_num'].count()
lrdf['number_of_rentals'] = lw.values
lrdf.head(5)

#rentals per bike
lrdf['per_bike'] = lrdf['number_of_rentals'] / lrdf['bikes_moving']
lrdf.head(10)

,data,bikes_moving,number_of_rentals,per_bike
0,2018-04-01,39,44,1.128205
1,2018-04-02,150,249,1.660000
2,2018-04-03,188,476,2.531915
3,2018-04-04,170,510,3.000000
4,2018-04-05,191,629,3.293194
5,2018-04-06,182,541,2.972527
6,2018-04-07,238,851,3.575630
7,2018-04-08,237,978,4.126582
8,2018-04-09,205,936,4.565854
9,2018-04-10,208,930,4.471154


In [64]:
#it means that:
print('One bike has been rented on average {0} a day'.format(lrdf['per_bike'].mean()))
print('The median of rentals for one bike is equal to {0}'.format(lrdf['per_bike'].median()))

One bike has been rented on average 3.1321611712981032 a day
The median of rentals for one bike is equal to 3.23579159787195


In [65]:
#what was the longest ride?
wyp2018[wyp2018['distance'] == wyp2018['distance'].max()].head(1)

,bike_num,start_time,end_time,departure,return,duration_sec,start_day,start_month,start_hour,start_minute,...,is_free,shop,route,dep_id,dep_lat,dep_lon,ret_id,ret_lat,ret_lon,distance
171347,58811,2018-06-05 14:51:29,2018-06-05 15:36:29,Al. Księżnej Jadwigi Śląskiej,Podlesie – Stary Most,2700,5,6,14,51,...,0,2,Al. Księżnej Jadwigi Śląskiej - Podlesie – Sta...,5872,50.284196,18.96789,5857,50.174393,18.970738,12.215416


<h2>Let's now focus on stations themselves.

In [66]:
#the stations 2018 dataset were read earlier, we can inspect it now
stations.head(3)

,name,id,lat,lon,capacity
0,COP24,5868,50.265282,19.028635,12
1,Dolina 3-ch Stawów,5869,50.233217,19.027812,12
2,Katowice Rynek,5867,50.258697,19.021618,24


In [67]:
#We can say that:
print('There were {0} stations in Katowice in 2018.'.format(stations['name'].count()))

There were 54 stations in Katowice in 2018.


In [68]:
#what is the distance between each departure station and city center?
#main square in Katowice: 50°15'34.7"N 19°01'19.6"E
wyp2018['centre_distance'] = wyp2018[['dep_lat','dep_lon']].apply(lambda x: geodesic( [ x[0], x[1] ] , [ 50.258697,19.021618] ).km, axis=1)

In [70]:
#plot stations on a map
import matplotlib.pyplot as plt
import mplleaflet

#Create a new figure.
stat_map = plt.figure(figsize=(20,10))
#plt.figure(figsize=(20,10))

#A scatter plot of *y* vs *x* with varying marker size and/or color.
plt.scatter(stations['lon'], stations['lat'], color='purple', s=50)

#Display a figure.
#plt.show()

#Convert a Matplotlib Figure to a Leaflet map. Embed in IPython notebook.
mplleaflet.display(fig=stat_map)

#Convert a Matplotlib Figure to a Leaflet map. Open in a browser
#mplleaflet.show(fig=stacje_mapa)

C:\Users\acibis\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
#how many bikes depart from and return to each station in summary?
dep = wyp2018.groupby(['departure'])['bike_num'].count()
ret = wyp2018.groupby(['return'])['bike_num'].count()
lats = stations.sort_values(by=['name'])['lat']
lons = stations.sort_values(by=['name'])['lon']
each_stat = pd.DataFrame(index = dep.index, data={ 'how_many_departs' : dep.values, 'how_many_returns' : ret.values,
                                           'lat' : lats.values, 'lon' : lons.values})
each_stat.head(5)

In [ ]:
#how many bikes depart from each station monthly?
departures = wyp2018.groupby(['departure','start_month'])['bike_num'].count()
deps_month=pd.DataFrame(departures)
deps_month = deps_month.unstack(level=1)
deps_month.head(5)

In [ ]:
#how many bikes return to each station monthly?
returns = wyp2018.groupby(['return','start_month'])['bike_num'].count()
reps_month=pd.DataFrame(returns)
reps_month = reps_month.unstack(level=1)
reps_month.head(5)

In [ ]:
#any shortages?
shortage = (deps_month - reps_month)
shortage['avg'] = shortage.mean(axis=1)
shortage[shortage['avg'] < 0].sort_values(by=['avg']).head(5)

In [566]:
#remeber to save your work!
wyp2018.to_csv('./dane/allrentals.csv')
stations.to_csv('./dane/allstations.csv')